In [1]:
import pandas as pd
import numpy as np

In [15]:
db_ranking = pd.read_csv('dbRanking.csv', delimiter='|')
db_ranking.sample(5)

,taxpayer_id,company_economic_activity_id,ciiu_spanish_desc,spanish_sector_company_name,employees_number,branches_number,activity_start_date,seniority_company_years_number,company_size_desc,foreign_company_mark_type,...,pbi_sector_2015_per,pbi_sector_2014_per,pbi_sector_2013_per,pbi_sector_2012_per,pbi_sector_2011_per,pbi_sector_2010_per,pbi_sector_2009_per,pbi_sector_2008_per,pbi_sector_2007_per,cutoff_date
1373,14205747548,7421,Actividades de arquitectura e ingenierÃ­a y ac...,CONSULTORIAS Y SERVICIOS,1.0,1.0,9999-12-31,18.0,PequeÃ±a empresa / Small-size Company,NaN,...,-3.2,-5.9,4.4,8.5,15.2,3.4,17.4,6.1,15.0,2020-02-28
1338,60012206079,7220,Consultores en programas de informÃ¡tica y sum...,INFORMATICA,37.0,1.0,1997-10-09,21.0,PequeÃ±a empresa / Small-size Company,NaN,...,3.9,0.0,0.0,0.0,0.0,9.2,8.2,1.3,11.5,2020-02-28
7893,95846892594,7414,Actividades de asesoramiento empresarial y mat...,CONSULTORIAS Y SERVICIOS,NaN,1.0,2009-03-02,9.0,PequeÃ±a empresa / Small-size Company,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-02-28
12147,1347513518,2520,FabricaciÃ³n de productos de plÃ¡stico,CAUCHO Y PLASTICO,NaN,NaN,1992-01-01,NaN,Mediana empresa / Medium-size Company,NaN,...,4.5,1.3,0.2,1.8,5.2,2.8,4.1,0.9,7.4,2020-02-28
777,70302899473,7414,Actividades de asesoramiento empresarial y mat...,CONSULTORIAS Y SERVICIOS,30.0,1.0,1982-02-08,36.0,Mediana empresa / Medium-size Company,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-02-28


In [16]:
ranking_dict = ['ranking_'+str(i)+'_number' for i in range(2017,2009,-1)]
min_usd_billing_dict = ['min_usd_billing_'+str(i)+'_amount' for i in range(2017,2012,-1)]
max_usd_billing_dict = ['max_usd_billing_'+str(i)+'_amount' for i in range(2017,2012,-1)]
min_pen_billing_dict = ['min_pen_billing_'+str(i)+'_amount' for i in range(2017,2012,-1)] 
max_pen_billing_dict = ['max_pen_billing_'+str(i)+'_amount' for i in range(2017,2012,-1)]
pbi_sector_dict = ['pbi_sector_'+str(i)+'_per' for i in range(2016,2006,-1)]

def clean_db_ranking(df):
    df = df.drop_duplicates(ignore_index=True)
    df = df.dropna(axis=1, how='all')
    df = df[df['market_share_per'] < 99] # Removing, these are like 10 registers
    df = df.drop(['ebitda_2014_amount', 'ebitda_2013_amount'], axis=1) # Removing these columns because are too few
    
    for col in ranking_dict: #computing and droping ranking
        df['ranking_2017_number'].fillna(df[col], inplace=True)
    df['ranking_last_number'] = df['ranking_2017_number']
    df = df.drop(ranking_dict, axis=1)
    
    for col in min_usd_billing_dict: #computing and dropping min usd amounts
        df['min_usd_billing_2017_amount'].fillna(df[col], inplace=True)
    df['min_usd_billing_last_amount'] = df['min_usd_billing_2016_amount']
    df = df.drop(min_usd_billing_dict, axis=1)

    for col in max_usd_billing_dict: #computing and dropping max usd amounts
        df['max_usd_billing_2017_amount'].fillna(df[col], inplace=True)
    df['max_usd_billing_last_amount'] = df['max_usd_billing_2017_amount']
    df = df.drop(max_usd_billing_dict, axis=1)

    for col in min_pen_billing_dict: #computing and dropping max usd amounts
        df['min_pen_billing_2017_amount'].fillna(df[col], inplace=True)
    df['min_pen_billing_last_amount'] = df['min_pen_billing_2017_amount']
    df = df.drop(min_pen_billing_dict, axis=1)

    for col in max_pen_billing_dict: #computing and dropping max usd amounts
        df['max_pen_billing_2017_amount'].fillna(df[col], inplace=True)
    df['max_pen_billing_last_amount'] = df['max_pen_billing_2017_amount']
    df = df.drop(max_pen_billing_dict, axis=1)

    df['exporter_mark_type_flag'] = df['exporter_mark_type']
    df['importer_mark_type_flag'] = df['importer_mark_type']

    df = df.drop(df.columns[[i for i in range(15, 185)]], axis=1)

    for col in pbi_sector_dict: #computing and dropping max usd amounts
        df['pbi_sector_2016_per'].fillna(df[col], inplace=True)
    df['pbi_sector_last_per'] = df['pbi_sector_2016_per']
    df['pbi_sector_last_per'] = df['pbi_sector_last_per'].replace(0, np.nan)
    df = df.drop(pbi_sector_dict, axis=1)

    df = df.drop(['cutoff_date'], axis=1)

    return df

In [17]:
db_ranking = clean_db_ranking(db_ranking)
# db_ranking.drop_duplicates()

In [19]:
db_ranking.to_excel('db_ranking_clean.xlsx')

In [6]:
db_ranking['spanish_sector_company_name'].value_counts().index.tolist()

['COMERCIALIZADORAS',
 'CONSULTORIAS Y SERVICIOS',
 'CONSTRUCTORAS',
 'TRANSPORTE',
 'METALMECANICA',
 'AGROINDUSTRIA Y GANADERIA',
 'INMOBILIARIAS',
 'TURISMO',
 'OTROS SERVICIOS',
 'HIDROCARBUROS',
 'INFORMATICA',
 'VEHICULOS',
 'TEXTILES',
 'MINERIA',
 'INDUSTRIA ALIMENTARIA',
 'EDUCACION',
 'CAUCHO Y PLASTICO',
 'TELECOMUNICACIONES',
 'PESCA',
 'SALUD',
 'ALQUILERES (MAQUINARIA)',
 'INTERMEDIACION FINANCIERA',
 'ESPARCIMIENTO',
 'AUTOMOTRIZ',
 'ABOGADOS',
 'QUIMICAS',
 'IMPRESIÃ“N',
 'MINERALES NO METALICOS',
 'FORESTAL Y SILVICULTURA',
 'ORGANIZACIONES',
 'ENERGIA Y AGUA',
 'EDITORIALES',
 'ADMINISTRACION PUBLICA',
 'PAPEL',
 'CURTIDURIA',
 'CORREOS',
 'RECICLAJE',
 'LABORATORIOS',
 'BEBIDAS',
 'INVESTIGACION Y DESARROLLO',
 'VETERINARIAS']

In [9]:
db_ranking['ciiu_spanish_desc'].value_counts()

ConstrucciÃ³n de edificios completos o de partes de edificios, obras de ingenierÃ­a civil       538
Venta al por mayor de otros productos                                                           522
Otras actividades empresariales n.c.p.                                                          349
Transporte de carga por carretera                                                               294
Actividades de arquitectura e ingenierÃ­a y actividaes conexas de asesoramiento tÃ©cnico        222
                                                                                               ... 
ReproducciÃ³n de grabaciones                                                                      1
Actividades de fotografÃ­a                                                                        1
FabricaciÃ³n de cojines, engranajes, trenes de engranajes y piezas de transmisiÃ³n                1
FabricaciÃ³n de artÃ­culos de cuchillerÃ­a, herramientas de mano y artÃ­culos de ferreterÃ­a      1


In [10]:
db_ranking['company_size_desc'].value_counts()

PequeÃ±a empresa / Small-size Company    3093
Mediana empresa / Medium-size Company    2648
Gran Empresa / Big-size Company            28
Name: company_size_desc, dtype: int64

In [17]:
db_ranking['department_name'].value_counts()

Lima                4167
La Libertad          190
Arequipa             176
Piura                131
Ancash               126
Ucayali              121
Lambayeque            98
JunÃ­n                83
Cajamarca             81
Loreto                77
Cusco                 75
San MartÃ­n           68
Puno                  64
HuÃ¡nuco              64
Ica                   62
Ayacucho              40
Tacna                 33
Madre de Dios         25
Tumbes                23
Pasco                 23
Amazonas              22
Moquegua              11
ApurÃ­mac              9
Huancavelica           3
Callao                 2
Av. JosÃ© GÃ¡lve       2
Madre De Dios          1
Name: department_name, dtype: int64